In [1]:
import gzip
import pandas as pd
import os
import random
import numpy as np
from sklearn.pipeline import Pipeline
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, classification_report, r2_score
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

In [2]:
import gensim.models as g
import codecs

In [3]:
def load_initial_data(path):
    df = []
    for line in gzip.open(path, 'rb'):
        df.append(eval(line))
    return df

In [4]:
data = []
for file in os.listdir("amazon"):
    data+=load_initial_data("amazon/"+file)

In [5]:
data = data[:int(len(data)*0.1)]
random.shuffle(data)
dataframe = pd.DataFrame.from_dict(data)

In [6]:
dataX, dataY = dataframe["question"], dataframe["questionType"]
dataY = dataY.apply(lambda y: 1 if y == "yes/no" else 0).as_matrix()


/home/ostapkharysh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [7]:
questions =list()
for el in range(len(dataframe)):
    questions.append(TaggedDocument(dataX[el], [dataY[el]]))

In [8]:
question_vecs = list()
model = g.Doc2Vec()
model.build_vocab(questions)
for el in range(len(dataX)):
    question_vecs.append((model.infer_vector(dataX[el])))

In [10]:
index = int(len(dataframe)*0.8) 

X_train = question_vecs[:index]
X_test = question_vecs[index:]

Y_train = dataY[:index]
Y_test = dataY[index:]

In [11]:
len(X_train)

111751

In [12]:
len(X_test)

27938

In [13]:
Y_train

array([0, 0, 1, ..., 0, 0, 0])

In [19]:
len(Y_test)

array([1, 1, 0, ..., 0, 0, 1])

In [15]:
mdl_lin = linear_model.LinearRegression()
mdl_log = linear_model.LogisticRegression()

In [16]:
mdl_log.fit(X_train, Y_train)
mdl_lin.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
pred_log = mdl_log.predict(X_test)
pred_lin = mdl_lin.predict(X_test)

In [18]:
print("LINEAR REGRESSION")
print("Accuracy: %.5f" % mdl_lin.score(X_test, Y_test))
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, pred_lin))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, pred_lin))
print()

print("LOGISTIC REGRESSION")
print("Accuracy: %.5f" % mdl_log.score(X_test, Y_test))
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, pred_log))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, pred_log))

LINEAR REGRESSION
Accuracy: 0.00127
Mean squared error: 0.25
Variance score: 0.00

LOGISTIC REGRESSION
Accuracy: 0.54438
Mean squared error: 0.46
Variance score: -0.84
